In [150]:
# PUT CUSTOM FUNCTIONS AT TOP SO WE CAN TRANSFER TO A SEPARATE PYTHON FILE

def get_metrics_smoted(model):
    '''
    model = instantiated instace of classifier
    '''
   
    model.fit(X_tr_sm, y_tr_sm)
    y_pred = model.predict(X_te_final_sc)
    
    return print(confusion_matrix(y_test, y_pred), '\n', 
                 classification_report(y_test, y_pred))

In [152]:
# copy & paste this cell in new notebook 
# to have all the libraries we're working with
# and the og data, modified df, X (feature vars) and y (target var)

# Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from operator import itemgetter # for sorting a list of tuples based on the second element of ea tuple.
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE

In [148]:
# importing ohe'd & scaled X_train/X_test as dataframes
# importing y_train and test as series

X_tr_final_sc = pd.read_csv('../../src/data/X_tr_final_sc_seeded_42.csv', index_col=0)
X_te_final_sc = pd.read_csv('../../src/data/X_te_final_sc_seeded_42.csv', index_col=0)

y_train = pd.read_csv('../../src/data/y_train_seeded_42.csv', index_col=0, squeeze=True)
y_test = pd.read_csv('../../src/data/y_test_seeded_42.csv', index_col=0, squeeze=True)

# print('train datasets: ', X_tr_final_sc.shape, y_train.shape)
# print('test datasets: ', X_te_final_sc.shape, y_test.shape)


# SMOTE

sm = SMOTE(random_state=42)
X_tr_sm, y_tr_sm = sm.fit_resample(X_tr_final_sc, y_train)

# print("SMOTE'd train datasets: ", X_tr_final_smoted.shape, y_tr_smoted.shape)
# print(y_tr_smoted.value_counts())

In [195]:
y_train.value_counts()

False    2141
True      358
Name: churn, dtype: int64

In [199]:
y_train

367     False
3103    False
549     False
2531    False
2378    False
        ...  
1095    False
1130    False
1294    False
860     False
3174    False
Name: churn, Length: 2499, dtype: bool

In [198]:
type(y_train)

pandas.core.series.Series

In [197]:
type(y_tr_sm)

pandas.core.series.Series

---

# Logistic Regression

#### ---

## log1
- FSM

In [181]:
log1 = LogisticRegression()
log1.fit(X_tr_final_sc, y_train)
log1_y_preds = log1.predict(X_te_final_sc)

print(confusion_matrix(y_test, log1_y_preds))

print(classification_report(y_test, log1_y_preds))

[[689  20]
 [ 97  28]]
              precision    recall  f1-score   support

       False       0.88      0.97      0.92       709
        True       0.58      0.22      0.32       125

    accuracy                           0.86       834
   macro avg       0.73      0.60      0.62       834
weighted avg       0.83      0.86      0.83       834



#### ---

## log2
- w/ SMOTE'd data

In [187]:
log2 = LogisticRegression()
get_metrics_smoted(log2)
cross_val_score(log2, X_tr_sm, y_tr_sm, cv=5).mean()

[[557 152]
 [ 28  97]] 
               precision    recall  f1-score   support

       False       0.95      0.79      0.86       709
        True       0.39      0.78      0.52       125

    accuracy                           0.78       834
   macro avg       0.67      0.78      0.69       834
weighted avg       0.87      0.78      0.81       834



0.7905238879377092

In [188]:
log2.coef_

array([[ 0.0907111 ,  0.14918545,  0.37633403,  0.02832051,  0.38485303,
         0.21320989,  0.02956331,  0.17232597,  0.09150291, -0.01154218,
         0.05790528,  0.08270365, -0.26559763,  0.19229876,  0.93016493,
        -0.34207872, -0.10569051, -0.26073038, -0.00338121, -0.17484781,
        -0.11327185, -0.15760381, -0.14993327, -0.10502922, -0.10753257,
        -0.30435305, -0.34194717, -0.18976325, -0.32579588, -0.10261089,
        -0.12792588, -0.11285494, -0.2240096 , -0.07531357, -0.13193443,
        -0.06432111, -0.11972284, -0.11603112, -0.17553953,  0.02893337,
         0.06349107, -0.06564975, -0.32434916, -0.09589287, -0.15235346,
        -0.04410733, -0.1551582 , -0.06519476, -0.08497626, -0.05154529,
        -0.1001378 , -0.10894069, -0.08217477, -0.25403555,  0.01839349,
        -0.15794845, -0.21225145, -0.01505115, -0.13359109, -0.36800088,
        -0.31457735, -0.08516023, -0.16404396, -0.14988084, -0.22858396,
         0.87621208, -0.6557467 ]])

In [189]:
lg2_summary = list(zip(X_tr_sm.columns, lg2.coef_[0]))
lg2_summary = sorted(lg2_summary,key=itemgetter(1))
lg2_summary

[('voice mail plan_yes', -0.5730842683785637),
 ('total intl calls', -0.22961469005355573),
 ('state_ND', -0.16473568335655644),
 ('state_VT', -0.14143289011214685),
 ('state_IA', -0.10904592391792269),
 ('state_IL', -0.07828570076407303),
 ('state_AL', -0.07802766681754467),
 ('state_VA', -0.06266473296042176),
 ('state_HI', -0.05026542906483044),
 ('state_AZ', -0.04379472711742875),
 ('state_RI', -0.031304693903095586),
 ('total night calls', -0.005950679236936779),
 ('state_CO', -0.004485061770315777),
 ('state_LA', 0.0014333874652364637),
 ('state_TN', 0.0015021579528435379),
 ('state_WI', 0.003809824905052089),
 ('state_SD', 0.008070007369558478),
 ('state_WY', 0.010918373083625608),
 ('total day calls', 0.013942660724815559),
 ('state_MO', 0.01599381339596603),
 ('state_DE', 0.023020432911934847),
 ('state_OK', 0.02531391079800895),
 ('total eve calls', 0.0271732930308543),
 ('state_NM', 0.029632886527504787),
 ('state_FL', 0.029805727924452616),
 ('account length', 0.03063228998

### ---

## log3
- w/ SMOTE'd data
- param tuned with gridsearch

In [184]:
log3 = LogisticRegression()

log3_param_grid = {
    'C': np.logspace(-3, 3, 20),
    'penalty': ['l1'],
    'solver': ['liblinear']
}

log3_gridsearch = GridSearchCV(estimator=log3, param_grid=log3_param_grid, cv=5)

log3_gridsearch.fit(X_tr_sm, y_tr_sm)

print(log3_gridsearch.best_params_, '\n', log3_gridsearch.best_score_)

{'C': 0.07847599703514611, 'penalty': 'l1', 'solver': 'liblinear'} 
 0.7947262238410452


In [190]:
log3_grd = LogisticRegression(C= 0.07847599703514611, penalty= 'l1', solver= 'liblinear')
get_metrics_smoted(log3_grd)

[[554 155]
 [ 29  96]] 
               precision    recall  f1-score   support

       False       0.95      0.78      0.86       709
        True       0.38      0.77      0.51       125

    accuracy                           0.78       834
   macro avg       0.67      0.77      0.68       834
weighted avg       0.87      0.78      0.81       834

